In [1]:
!ls -a

.                   .env                .ipynb_checkpoints  client_secrets.json
..                  .git                README.md           setting.txt
.DS_Store           .gitignore          Untitled.ipynb      test.ipynb


In [2]:
import pandas as pd

In [4]:
import os
from os.path import join, dirname
from dotenv import load_dotenv
from pathlib import Path

In [5]:
load_dotenv(verbose=True)
dotenv_path = join(Path().resolve(), '.env')
load_dotenv(dotenv_path)

True

In [9]:
GOOGLE_DRIVE_PATH = os.environ.get("GOOGLE_DRIVE_PATH")

In [8]:
pd.read_csv(GOOGLE_DRIVE_PATH + '/horse_racing/data/test.csv')

,aaa,aaa.1,dd,tge,ge
